RAG PIPELINE IMPROVEMENT 

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load your .env file containing GROQ_API_KEY


True

1] DATA INGECTION

In [ ]:

from langchain_community.document_loaders import TextLoader, PyPDFLoader, WebBaseLoader

# PDF data injection
pdf_loader = PyPDFLoader("DeepLearning.pdf")
pdf_docs = pdf_loader.load()

# Text data injection
text_loader = TextLoader("MachineLearning.txt")
text_docs = text_loader.load()

# Web data injection
web_loader = WebBaseLoader("https://www.geeksforgeeks.org/artificial-intelligence/what-is-generative-ai/")
web_docs = web_loader.load()

# Combine all
all_docs = pdf_docs + text_docs + web_docs


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(all_docs)


3] DATA EMBEDDING AND VECTOR STORE

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_store = FAISS.from_documents(documents, embedding_model) 
vector_store.save_local("faiss_index")


In [11]:
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Fetch the GROQ API key from the environment
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize the LLM using the API key from .env
llm = ChatGroq(
    temperature=0,
    model_name="llama3-8b-8192",
    groq_api_key=groq_api_key
)

# Load FAISS index (assumes you already created faiss_index with the same embedding model)
vector_store = FAISS.load_local(
    "faiss_index", 
    embedding_model, 
    allow_dangerous_deserialization=True
)


In [12]:
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

prompt_template = """
Use the context below to answer the question.
If you don't know the answer, just say "I don't know".

Context:
{context}

Question:
{question}

Helpful Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)


In [14]:
query = "What is Machine Learning?"
response = qa_chain.run(query)

print("Answer:", response)


Answer: Machine learning is a branch of Artificial Intelligence that focuses on developing models and algorithms that let computers learn from data without being explicitly programmed for every task. In simple words, ML teaches the systems to think and understand like humans by learning from the data.
